In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('C:\\Users\\sgibson\\Desktop\\GMU\\CS504\\Group_Project\\Hospital_Inpatient_Cost_Transparency__Beginning_2009_JOINED.csv')

In [16]:
# Weighted Average Charge by Year
def wavg_year(data, years):
    z = []
    for i in years:
        y = data[data['Year']==i]
        y_wavg = (y["Mean Charge"] * y["Discharges"]).sum() / y["Discharges"].sum()
        z.append({'Year': i, 'Avg Charge': round(y_wavg, 2), 'Total Discharges': (y["Discharges"].sum())})

    return pd.DataFrame(z)

# Weighted Average Charge by Severity
def wavg_sev(data, severity):
    x = []
    for i in severity.keys():
        s = data[data['APR Severity of Illness Code']==i]
        s_wavg = (s["Mean Charge"] * s["Discharges"]).sum() / s["Discharges"].sum()
        x.append({'Severity': severity[i], 'Avg Charge': round(s_wavg, 2), 'Total Discharges': (s["Discharges"].sum())})

    return pd.DataFrame(x)

# Price Comparison final output
def price_comp(data, county, DRG, savepath):
    # sorted_year = Sort data by year to find most current year
    sorted_year = df.sort_values(by='Year', ascending=False)

    # current_year = value with location 0,0 in sorted dataframe
    current_year = sorted_year.iloc[0,0]

    # statewide_severity = Get all rows where year equals 'current_year' and 'DRGcode' 
    statewide_severity = data[(data['Year']==current_year) & (data['APR DRG Code']==DRG)]

    # countywide_severity = From statewide where county equals county entered by user
    countywide_severity = statewide_severity[statewide_severity['County']==county]

    # Define severity
    severity = {1: 'Minor', 2: 'Moderate', 3: 'Major', 4: 'Extreme'}

    ### returns weighted average by severity at county level
    county_avg_by_severity = wavg_sev(countywide_severity, severity)

    ### returns weighted average by severity at state level
    state_avg_by_severity = wavg_sev(statewide_severity, severity)

    # define year
    years = df['Year'].unique()

    # statewide = Get all rows where year equals 'DRGcode' 
    statewide = df[(df['APR DRG Code']==DRG)]

    # countywide = From statewide where county equals county entered by user
    countywide = statewide[statewide['County']==county]

    ### returns weighted average by year at state level
    state_avg_by_year = wavg_year(statewide, years)

    ###returns weighted average by year at county level
    county_avg_by_year = wavg_year(countywide, years)

    print('\n Average charge per year at state level')
    print(state_avg_by_year)
    print('\n Average charge per year at county level')
    print(county_avg_by_year)
    print('\n Average charge per severity at state level')
    print(state_avg_by_severity)
    print('\n Average charge per severity at county level')
    print(county_avg_by_severity)
             
    state_avg_by_year.to_csv(r'{}{}_NY_avg_by_year.csv'.format(savepath, DRG))
    county_avg_by_year.to_csv(r'{}{}_{}_county_avg_by_year.csv'.format(savepath, DRG, county))
    state_avg_by_severity.to_csv(r'{}{}_NY_avg_by_severity.csv'.format(savepath, DRG))
    county_avg_by_severity.to_csv(r'{}{}_{}_county_avg_by_severity.csv'.format(savepath, DRG, county))
    

In [17]:
## User input##
# DRG code of interest
DRG = 70
# County of interest
county = 'Bronx'
#enter location to save csv files
savepath = 'C:\\Users\\sgibson\\Desktop\\GMU\\CS504\\Group_Project\\'
#function to display final price comparison info
price_comp(df, county, DRG, savepath)


 Average charge per year at state level
   Year  Avg Charge  Total Discharges
0  2016    48909.62               312
1  2015    42357.34               401
2  2014    40542.35               460
3  2013    40623.75               477
4  2012    31873.10               474
5  2011    30010.75               541
6  2010    25860.58               544
7  2009    28924.62               606

 Average charge per year at county level
   Year  Avg Charge  Total Discharges
0  2016    56958.40                21
1  2015    28893.12                17
2  2014    29012.34                34
3  2013    39102.54                36
4  2012    30909.88                54
5  2011    25186.86                46
6  2010    20299.40                60
7  2009    30828.34                74

 Average charge per severity at state level
   Severity  Avg Charge  Total Discharges
0     Minor    34311.84                97
1  Moderate    45855.29               169
2     Major    90927.40                43
3   Extreme    90709

C:\Users\sgibson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
